In [92]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#### 1.设置输入和输出节点的个数,配置神经网络的参数。

In [93]:
INPUT_NODE = 784     # 输入节点
OUTPUT_NODE = 10     # 输出节点
LAYER1_NODE = 500    # 隐藏层数       
                              
BATCH_SIZE = 100     # 每次batch打包的样本个数        

# 模型相关的参数
LEARNING_RATE_BASE = 0.8      
LEARNING_RATE_DECAY = 0.99    
REGULARAZTION_RATE = 0.0001   
TRAINING_STEPS = 3000        
MOVING_AVERAGE_DECAY = 0.99  

#### 2. 定义辅助函数来计算前向传播结果，使用ReLU做为激活函数。

In [94]:
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 不使用滑动平均类
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        return tf.matmul(layer1, weights2) + biases2

    else:
        # 使用滑动平均类，avg_class.average(weights1) 获取滑动平均后变量weights1的值
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)  

def inference1(input_tensor, avg_class, reuse=False):
    if avg_class == None:
        # 定义第一层神经网络的变量和前向传播过程
        with tf.variable_scope('layer1', reuse=reuse):
            weights = tf.get_variable("weights", [INPUT_NODE, LAYER1_NODE],
                                     initializer=tf.truncated_normal_initializer(stddev=0.1))
            biases = tf.get_variable("biases", [LAYER1_NODE],
                                    initializer=tf.constant_initializer(0.0))
            layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
        # 类似定义输出层变量和前向传播过程
        with tf.variable_scope('layer2', reuse=reuse):
            weights = tf.get_variable("weights", [LAYER1_NODE, OUTPUT_NODE],
                                     initializer=tf.truncated_normal_initializer(stddev=0.1))
            biases = tf.get_variable("biases", [OUTPUT_NODE],
                                    initializer=tf.constant_initializer(0.0))
            layer2 = tf.matmul(layer1, weights) + biases
    else:
        # 定义第一层神经网络的变量和前向传播过程
        with tf.variable_scope('layer1', reuse=reuse):
            weights = tf.get_variable("weights", [INPUT_NODE, LAYER1_NODE],
                                     initializer=tf.truncated_normal_initializer(stddev=0.1))
            biases = tf.get_variable("biases", [LAYER1_NODE],
                                    initializer=tf.constant_initializer(0.0))
            layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights)) + avg_class.average(biases))
        # 类似定义输出层变量和前向传播过程
        with tf.variable_scope('layer2', reuse=reuse):
            weights = tf.get_variable("weights", [LAYER1_NODE, OUTPUT_NODE],
                                     initializer=tf.truncated_normal_initializer(stddev=0.1))
            biases = tf.get_variable("biases", [OUTPUT_NODE],
                                    initializer=tf.constant_initializer(0.0))
            layer2 = tf.matmul(layer1, avg_class.average(weights)) + avg_class.average(biases)
    
    return layer2

#### 3. 定义训练过程。

In [119]:
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input') # 输出为10个数字
   
    # 计算不含滑动平均类的前向传播结果
    y = inference1(x, None, reuse=True)
    
    # 定义训练轮数及相关的滑动平均类 
    global_step = tf.Variable(0, trainable=False)
    # 定义滑动平均类
#     variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
#     variables_averages_op = variable_averages.apply(tf.trainable_variables())
#     average_y = inference1(x, variable_averages, True)
    
    # 计算交叉熵及其平均值
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 损失函数的计算
    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    with tf.variable_scope('', reuse=True):
        regularaztion = regularizer(tf.get_variable('layer1/weights')) + \
                                    regularizer(tf.get_variable('layer2/weights'))
    loss = cross_entropy_mean + regularaztion
    
    # 设置指数衰减的学习率。
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)
    
    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # 反向传播更新参数和更新每一个参数的滑动平均值
    # 为了一次完成多个操作，TensorFLow提供了tf.control_dependencies和tf.group两种机制
    #with tf.control_dependencies([train_step, variables_averages_op]):
    #    train_op = tf.no_op(name='train')

    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化会话并开始训练过程。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        test_feed = {x: mnist.test.images, y_: mnist.test.labels} 
        
        # 循环的训练神经网络。
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                test_acc=sess.run(accuracy,feed_dict=test_feed)
                print("After %d training step(s), validation accuracy using average model is %g, test accuracy is %g " % (i,validate_acc, test_acc))
                print("lrate:%s" % sess.run(learning_rate))
                
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_step,feed_dict={x:xs,y_:ys})

        test_acc=sess.run(accuracy,feed_dict=test_feed)
        print(("After %d training step(s), test accuracy using average model is %g" %(TRAINING_STEPS, test_acc)))
   

#### 4. 主程序入口，这里设定模型训练次数为5000次。

In [120]:
def main(argv=None):
    mnist = input_data.read_data_sets("../MNIST_data/", one_hot=True)
    train(mnist)

if __name__=='__main__':
    main()

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
After 0 training step(s), validation accuracy using average model is 0.0712, test accuracy is 0.0689 
lrate:0.8
After 1000 training step(s), validation accuracy using average model is 0.9758, test accuracy is 0.9729 
lrate:0.792
After 2000 training step(s), validation accuracy using average model is 0.9778, test accuracy is 0.9779 
lrate:0.776239
After 3000 training step(s), test accuracy using average model is 0.9818
